# Telegram Market Screener Report Generator (Cache-Based)

## 📊 Overview
This notebook generates automated market screener reports using cached CLOB data and distributes them via Telegram. It integrates with the QuantsLab notification system to send formatted volume reports, interactive charts, and detailed data files directly to Telegram channels or users for real-time market monitoring.

## 🎯 Objectives
1. **Data Loading**: Load cached candle data from CLOB data source
2. **Volume Analysis**: Calculate volume metrics and identify top performers
3. **Report Generation**: Create formatted volume reports with rankings
4. **Telegram Integration**: Send reports via Telegram bot using notification system
5. **Chart Creation**: Generate interactive Plotly charts for visual analysis
6. **File Distribution**: Send CSV reports and chart files for detailed analysis

## 📋 Prerequisites
- Cached CLOB candle data (run data collection tasks first)
- Telegram bot configuration (bot token and chat ID in .env)
- QuantsLab notification system setup
- Environment variables configured (.env file)
- Plotly and pandas for analysis

## ⚠️ Important Notes
- **Data Source**: Uses cached CLOB data instead of live database queries
- **Telegram Bot Setup**: Requires BotFather bot token and target chat ID
- **File Handling**: Temporary files created in /tmp directory for chart/CSV export
- **Rate Limiting**: Be mindful of Telegram API rate limits for file uploads

## 📈 Expected Outputs
- Formatted volume report messages sent to Telegram
- Interactive volume charts (PNG and HTML formats)
- Detailed CSV reports with complete market data
- Real-time market monitoring capabilities via Telegram bot

In [1]:
# 🔌 Initialize QuantsLab Services
# Import and configure core services and notification system
import warnings
warnings.filterwarnings("ignore")

from core.data_sources.clob import CLOBDataSource
from core.notifiers import NotificationManager, NotificationMessage
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
from datetime import datetime, timezone
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ QuantsLab modules imported successfully")
print("📊 Services available: CLOB data source, Notification system")
print("⚙️ Environment variables loaded from .env")

✅ QuantsLab modules imported successfully
📊 Services available: CLOB data source, Notification system
⚙️ Environment variables loaded from .env


# ⚙️ Configuration and Data Loading
# Set up data source and load cached market data

In [2]:
# Configuration Parameters
CONNECTOR_NAME = "binance_perpetual"  # Exchange connector to use
QUOTE_ASSET = "USDT"                  # Quote currency for trading pairs
INTERVAL = "15m"                       # Candle interval for analysis
FETCH_FRESH_DATA = False              # Set to True to fetch fresh data from exchange
DAYS = 7                              # Days of historical data (when fetching fresh)
TOP_PAIRS_COUNT = 15                  # Number of top pairs to include in report

# Initialize CLOB data source
clob = CLOBDataSource()

# Initialize notification manager
notification_manager = NotificationManager()
enabled_notifiers = notification_manager.get_enabled_notifiers()

print(f"📊 Configuration loaded:")
print(f"  - Connector: {CONNECTOR_NAME}")
print(f"  - Quote Asset: {QUOTE_ASSET}")
print(f"  - Interval: {INTERVAL}")
print(f"  - Top Pairs: {TOP_PAIRS_COUNT}")
print(f"🔔 Enabled notifiers: {', '.join(enabled_notifiers) if enabled_notifiers else 'None configured'}")

📊 Configuration loaded:
  - Connector: binance_perpetual
  - Quote Asset: USDT
  - Interval: 15m
  - Top Pairs: 15
🔔 Enabled notifiers: telegram


In [3]:
# 📊 Load Market Data from Cache
# Load cached candle data or fetch fresh data if configured

if FETCH_FRESH_DATA:
    print("🔄 Fetching fresh data from exchange...")
    # Get trading pairs
    trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
    trading_pairs = [
        pair for pair in trading_rules.get_all_trading_pairs() 
        if pair.split("-")[1] == QUOTE_ASSET
    ]
    
    # Fetch candle data
    candles = await clob.get_candles_batch_last_days(
        CONNECTOR_NAME, trading_pairs, INTERVAL, DAYS, 2, 1.0
    )
    
    # Save to cache
    clob.dump_candles_cache()
    print(f"✅ Fetched data for {len(candles)} trading pairs")
else:
    print("📂 Loading data from cache...")
    clob.load_candles_cache()
    print("✅ Cache loaded successfully")

# Filter cached data for our requirements
candles = [
    candle for key, candle in clob.candles_cache.items()
    if key[0] == CONNECTOR_NAME and key[2] == INTERVAL and key[1].endswith(f"-{QUOTE_ASSET}")
]

print(f"📈 Found {len(candles)} datasets matching criteria:")
print(f"  - Connector: {CONNECTOR_NAME}")
print(f"  - Interval: {INTERVAL}")
print(f"  - Quote Asset: {QUOTE_ASSET}")

if len(candles) == 0:
    print("⚠️  No cached data found. Please:")
    print("   1. Set FETCH_FRESH_DATA = True to fetch new data")
    print("   2. Or run data collection tasks to populate cache")
else:
    sample_pairs = [candle.trading_pair for candle in candles[:5]]
    print(f"📊 Sample pairs: {', '.join(sample_pairs)}{'...' if len(candles) > 5 else ''}")

📂 Loading data from cache...
✅ Cache loaded successfully
📈 Found 21 datasets matching criteria:
  - Connector: binance_perpetual
  - Interval: 15m
  - Quote Asset: USDT
📊 Sample pairs: XRP-USDT, ETH-USDT, DOGE-USDT, DOT-USDT, OP-USDT...


# 📊 Market Analysis and Volume Metrics
# Calculate volume metrics and create market ranking

In [4]:
# 💹 Calculate Volume Metrics for Market Screening
# Process each trading pair and calculate comprehensive volume metrics

volume_report = []

for candle in candles:
    try:
        df = candle.data.copy()
        
        if df.empty or len(df) < 24:  # Need at least 24 hours of data
            continue
            
        # Calculate volume metrics
        latest_price = df['close'].iloc[-1]
        
        # Volume analysis (last 24 hours)
        df_24h = df.tail(24) if len(df) >= 24 else df
        volume_24h_base = df_24h['volume'].sum()
        volume_24h_usd = volume_24h_base * latest_price
        
        # Price change analysis
        price_24h_ago = df_24h['close'].iloc[0] if len(df_24h) > 1 else latest_price
        price_change_24h = ((latest_price - price_24h_ago) / price_24h_ago * 100) if price_24h_ago > 0 else 0
        
        # Volatility analysis
        price_std = df_24h['close'].std()
        volatility_pct = (price_std / latest_price * 100) if latest_price > 0 else 0
        
        # High/Low analysis
        high_24h = df_24h['high'].max()
        low_24h = df_24h['low'].min()
        
        # Current position in 24h range (0 = at low, 1 = at high)
        range_24h = high_24h - low_24h
        position_in_range = ((latest_price - low_24h) / range_24h) if range_24h > 0 else 0.5
        
        # Volume trend (compare recent vs earlier volume)
        recent_volume = df.tail(6)['volume'].mean()  # Last 6 hours average
        earlier_volume = df.iloc[-24:-6]['volume'].mean() if len(df) >= 24 else recent_volume
        volume_trend = ((recent_volume - earlier_volume) / earlier_volume * 100) if earlier_volume > 0 else 0
        
        # Calculate a composite score (volume * volatility * price momentum)
        momentum_factor = abs(price_change_24h) / 100  # Normalize price change
        volume_factor = volume_24h_usd / 1000000  # Normalize to millions USD
        volatility_factor = volatility_pct / 100  # Normalize volatility
        
        composite_score = volume_factor * volatility_factor * (1 + momentum_factor)
        
        volume_report.append({
            'trading_pair': candle.trading_pair,
            'connector_name': CONNECTOR_NAME,
            'price': latest_price,
            'volume_24h_base': volume_24h_base,
            'volume_24h_usd': volume_24h_usd,
            'price_change_24h_pct': price_change_24h,
            'volatility_pct': volatility_pct,
            'high_24h': high_24h,
            'low_24h': low_24h,
            'position_in_range': position_in_range,
            'volume_trend_pct': volume_trend,
            'composite_score': composite_score,
            'data_points': len(df)
        })
        
    except Exception as e:
        print(f"⚠️  Error processing {candle.trading_pair}: {e}")
        continue

# Convert to DataFrame and sort by volume
df = pd.DataFrame(volume_report)

if df.empty:
    print("❌ No data could be processed for volume analysis")
    print("💡 Please check if cached data is available or fetch fresh data")
else:
    # Sort by volume (USD) and add ranking
    df = df.sort_values('volume_24h_usd', ascending=False).reset_index(drop=True)
    df['volume_rank'] = range(1, len(df) + 1)
    
    print(f"📊 Volume Analysis Complete:")
    print(f"  - Total pairs analyzed: {len(df)}")
    print(f"  - Top volume: ${df['volume_24h_usd'].max():,.0f}")
    print(f"  - Date range: {INTERVAL} candles")
    
    # Show top performers
    print(f"\n🏆 TOP {min(10, len(df))} BY VOLUME:")
    print("-" * 70)
    
    display_cols = ['volume_rank', 'trading_pair', 'volume_24h_usd', 'price_change_24h_pct', 'volatility_pct']
    top_pairs = df[display_cols].head(10)
    
    for _, row in top_pairs.iterrows():
        print(f"  {row['volume_rank']:2d}. {row['trading_pair']:12s} | "
              f"Vol: ${row['volume_24h_usd']:>10,.0f} | "
              f"Change: {row['price_change_24h_pct']:>+6.2f}% | "
              f"Vol: {row['volatility_pct']:>5.2f}%")

📊 Volume Analysis Complete:
  - Total pairs analyzed: 21
  - Top volume: $4,380,802,256
  - Date range: 15m candles

🏆 TOP 10 BY VOLUME:
----------------------------------------------------------------------
   1. ETH-USDT     | Vol: $4,380,802,256 | Change:  +0.80% | Vol:  0.26%
   2. BTC-USDT     | Vol: $3,864,052,880 | Change:  +0.36% | Vol:  0.19%
   3. SOL-USDT     | Vol: $1,624,360,790 | Change:  +1.17% | Vol:  0.44%
   4. WLD-USDT     | Vol: $1,260,812,640 | Change: +12.32% | Vol:  3.88%
   5. XRP-USDT     | Vol: $573,313,000 | Change:  +1.44% | Vol:  0.53%
   6. DOGE-USDT    | Vol: $489,970,725 | Change:  +1.88% | Vol:  0.44%
   7. ADA-USDT     | Vol: $225,605,574 | Change:  +1.99% | Vol:  0.61%
   8. LINK-USDT    | Vol: $185,966,285 | Change:  +1.46% | Vol:  0.38%
   9. AVAX-USDT    | Vol: $134,115,202 | Change:  +2.68% | Vol:  0.74%
  10. BNB-USDT     | Vol: $103,109,510 | Change:  +0.49% | Vol:  0.18%


# 📤 Generate and Send Telegram Volume Report
# Create formatted report and send via Telegram notification system

In [5]:
# 🚀 Generate Volume Report for Telegram Distribution
# Create comprehensive volume report with top performing pairs

if not df.empty:
    # Select top pairs for the report
    top_pairs = df.head(TOP_PAIRS_COUNT)
    
    # Generate report timestamp
    report_time = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
    
    # Calculate summary statistics
    total_volume = df['volume_24h_usd'].sum()
    avg_price_change = df['price_change_24h_pct'].mean()
    max_gainer = df.loc[df['price_change_24h_pct'].idxmax()]
    max_loser = df.loc[df['price_change_24h_pct'].idxmin()]
    
    # Build the report message
    report_message = f"""📊 <b>Volume Screener Report</b>
📅 {report_time}
🔗 Exchange: {CONNECTOR_NAME.replace('_', ' ').title()}
⏰ Timeframe: {INTERVAL} candles

📈 <b>Market Summary</b>
💰 Total Volume (Top {len(df)}): ${total_volume:,.0f}
📊 Avg Price Change: {avg_price_change:+.2f}%
🚀 Top Gainer: {max_gainer['trading_pair']} ({max_gainer['price_change_24h_pct']:+.2f}%)
📉 Top Loser: {max_loser['trading_pair']} ({max_loser['price_change_24h_pct']:+.2f}%)

🏆 <b>Top {TOP_PAIRS_COUNT} by Volume</b>
"""
    
    # Add top pairs to the report
    for i, (_, row) in enumerate(top_pairs.iterrows(), 1):
        # Format volume in appropriate units
        volume_usd = row['volume_24h_usd']
        if volume_usd >= 1_000_000:
            volume_str = f"${volume_usd/1_000_000:.1f}M"
        elif volume_usd >= 1_000:
            volume_str = f"${volume_usd/1_000:.0f}K"
        else:
            volume_str = f"${volume_usd:.0f}"
        
        # Emoji for price change
        change_pct = row['price_change_24h_pct']
        change_emoji = "🟢" if change_pct > 0 else "🔴" if change_pct < 0 else "⚪"
        
        # Format the line
        report_message += f"{i:2d}. <code>{row['trading_pair']:12s}</code> {volume_str:>8s} {change_emoji} {change_pct:+5.1f}%\n"
    
    # Add footer
    report_message += f"\n💡 <i>Data from {len(df)} trading pairs | QuantsLab</i>"
    
    print("📤 Volume report generated successfully")
    print(f"📊 Report includes {len(top_pairs)} top pairs")
    print(f"💰 Total market volume: ${total_volume:,.0f}")
    
else:
    report_message = f"""⚠️ <b>Volume Screener Report - No Data</b>
📅 {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}

❌ No market data available for analysis
💡 Please check data collection tasks or fetch fresh data

🔧 <i>QuantsLab Market Screener</i>"""
    
    print("⚠️  No data available for report generation")

# Display the report for preview
print("\n📋 TELEGRAM REPORT PREVIEW:")
print("=" * 50)
print(report_message.replace('<b>', '**').replace('</b>', '**').replace('<code>', '`').replace('</code>', '`').replace('<i>', '_').replace('</i>', '_'))
print("=" * 50)

📤 Volume report generated successfully
📊 Report includes 15 top pairs
💰 Total market volume: $13,370,789,540

📋 TELEGRAM REPORT PREVIEW:
📊 **Volume Screener Report**
📅 2025-09-10 13:37 UTC
🔗 Exchange: Binance Perpetual
⏰ Timeframe: 15m candles

📈 **Market Summary**
💰 Total Volume (Top 21): $13,370,789,540
📊 Avg Price Change: +2.26%
🚀 Top Gainer: WLD-USDT (+12.32%)
📉 Top Loser: BTC-USDT (+0.36%)

🏆 **Top 15 by Volume**
 1. `ETH-USDT    ` $4380.8M 🟢  +0.8%
 2. `BTC-USDT    ` $3864.1M 🟢  +0.4%
 3. `SOL-USDT    ` $1624.4M 🟢  +1.2%
 4. `WLD-USDT    ` $1260.8M 🟢 +12.3%
 5. `XRP-USDT    `  $573.3M 🟢  +1.4%
 6. `DOGE-USDT   `  $490.0M 🟢  +1.9%
 7. `ADA-USDT    `  $225.6M 🟢  +2.0%
 8. `LINK-USDT   `  $186.0M 🟢  +1.5%
 9. `AVAX-USDT   `  $134.1M 🟢  +2.7%
10. `BNB-USDT    `  $103.1M 🟢  +0.5%
11. `ARB-USDT    `   $81.3M 🟢  +3.1%
12. `LTC-USDT    `   $72.3M 🟢  +0.8%
13. `UNI-USDT    `   $70.1M 🟢  +2.6%
14. `NEAR-USDT   `   $65.9M 🟢  +2.8%
15. `DOT-USDT    `   $60.4M 🟢  +2.6%

💡 _Data from 21 tradin

In [6]:
# 🚀 Send Volume Report via Telegram
# Distribute the report through the notification system

notification = NotificationMessage(
    title="📊 Daily Volume Screener Report",
    message=report_message,
    level="info",
)

print("📤 Sending volume report via notification system...")

# Send notification to all enabled notifiers
try:
    results = await notification_manager.send_notification(notification)
    
    print("\n📊 Notification Delivery Results:")
    print("-" * 40)
    for service, success in results.items():
        status_emoji = "✅" if success else "❌"
        status_text = "Delivered" if success else "Failed"
        print(f"  {status_emoji} {service.capitalize()}: {status_text}")
    
    # Summary
    successful_deliveries = sum(results.values())
    total_services = len(results)
    print(f"\n📈 Delivery Summary: {successful_deliveries}/{total_services} services successful")
    
    if successful_deliveries == 0:
        print("⚠️ No notifications were delivered successfully")
        print("💡 Check your notification configuration in .env file")
    elif 'telegram' in results and results['telegram']:
        print("🎉 Volume report successfully sent to Telegram!")
    else:
        print("⚠️ Telegram delivery failed - check bot configuration")
        
except Exception as e:
    print(f"❌ Error sending notification: {e}")
    print("💡 Check your Telegram bot configuration and network connection")

📤 Sending volume report via notification system...

📊 Notification Delivery Results:
----------------------------------------
  ✅ Telegram: Delivered

📈 Delivery Summary: 1/1 services successful
🎉 Volume report successfully sent to Telegram!


# 📊 Advanced: Interactive Volume Charts
# Generate and send visual analytics via Telegram

In [7]:
# 🎨 Generate and Send Volume Chart as PNG
# Create professional volume visualization chart for Telegram

if not df.empty and len(df) >= 10:
    print("🎨 Creating volume chart for Telegram...")
    
    # Select top 10 pairs by volume for visualization
    top_10 = df.nlargest(10, 'volume_24h_usd')
    
    # Create horizontal bar chart
    fig = go.Figure()
    
    # Generate clean pair labels
    pair_labels = [row['trading_pair'] for _, row in top_10.iterrows()]
    
    # Create color scheme based on price change
    colors = ['#FF4444' if change < 0 else '#44FF44' if change > 0 else '#888888' 
              for change in top_10['price_change_24h_pct']]
    
    fig.add_trace(go.Bar(
        y=pair_labels,
        x=top_10['volume_24h_usd'],
        orientation='h',
        marker=dict(
            color=colors,
            opacity=0.8,
            line=dict(color='rgba(0,0,0,0.2)', width=1)
        ),
        text=[f"${vol/1e6:.1f}M" for vol in top_10['volume_24h_usd']],
        textposition='inside',
        textfont=dict(color='white', size=11, family='Arial Bold'),
        hovertemplate=(
            '<b>%{customdata[0]}</b><br>'
            'Volume: $%{x:,.0f}<br>'
            'Change: %{customdata[1]:+.2f}%<br>'
            'Price: $%{customdata[2]:.4f}'
            '<extra></extra>'
        ),
        customdata=list(zip(
            top_10['trading_pair'],
            top_10['price_change_24h_pct'],
            top_10['price']
        ))
    ))
    
    # Customize layout for clean PNG export
    fig.update_layout(
        title=dict(
            text=f"Top 10 Trading Pairs by 24h Volume<br><sub>{CONNECTOR_NAME.replace('_', ' ').title()} - {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}</sub>",
            x=0.5,
            xanchor='center',
            font=dict(size=18, family='Arial Bold', color='#333333')
        ),
        xaxis_title="24-Hour Volume (USD)",
        yaxis_title="",
        template="plotly_white",
        height=600,
        width=1000,
        margin=dict(l=120, r=80, t=100, b=60),
        xaxis=dict(
            tickformat='$,.0s',
            showgrid=True,
            gridwidth=1,
            gridcolor='#E0E0E0',
            tickfont=dict(size=10)
        ),
        yaxis=dict(
            showgrid=False,
            tickfont=dict(size=11, family='Arial')
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family='Arial', size=12, color='#333333')
    )
    
    # Display chart in notebook
    fig.show()
    
    # ============================================
    # SAVE AND SEND PNG CHART VIA TELEGRAM
    # ============================================
    chart_png_path = '/tmp/volume_chart.png'
    
    print("💾 Generating PNG chart for Telegram...")
    
    try:
        # Save as high-quality PNG
        fig.write_image(
            chart_png_path, 
            width=1000, 
            height=600, 
            scale=2,  # 2x resolution for better quality
            engine='kaleido'
        )
        print(f"✅ PNG chart saved: {chart_png_path}")
        
        # Send via Telegram
        telegram_notifier = notification_manager.get_notifier('telegram')
        
        if telegram_notifier:
            print("📤 Sending PNG chart via Telegram...")
            
            # Create caption for the image
            png_caption = f"""📊 <b>Volume Analysis Chart</b>

🏆 Top 10 trading pairs by 24h volume
💰 Total Volume: ${top_10['volume_24h_usd'].sum():,.0f}
📈 {CONNECTOR_NAME.replace('_', ' ').title()}

🟢 Green = Price Up | 🔴 Red = Price Down

<i>Generated by QuantsLab Market Screener</i>"""
            
            # Send PNG image
            chart_success = await telegram_notifier.send_photo(
                chart_png_path,
                caption=png_caption
            )
            
            print(f"  📊 PNG Chart: {'✅ Sent successfully!' if chart_success else '❌ Failed to send'}")
            
            if chart_success:
                print("🎉 Volume chart delivered to Telegram!")
            else:
                print("⚠️ Failed to send chart - check Telegram configuration")
                print(f"💾 Chart saved locally at: {chart_png_path}")
                
        else:
            print("⚠️ Telegram notifier not configured")
            print(f"💾 PNG chart saved locally: {chart_png_path}")
            
    except ImportError as e:
        print(f"❌ Kaleido not installed properly: {e}")
        print("💡 Try: pip install --upgrade kaleido")
        print("💡 Or: conda install -c conda-forge python-kaleido")
    except Exception as e:
        print(f"❌ Error generating PNG chart: {e}")
        print("💡 Make sure kaleido is properly installed")
        
        # Fallback: Save as HTML
        html_path = '/tmp/volume_chart.html'
        fig.write_html(html_path)
        print(f"📄 Saved as HTML fallback: {html_path}")
        
else:
    print("⚠️ Insufficient data to generate volume chart")
    if df.empty:
        print("💡 No data available - ensure data collection tasks are running")
    else:
        print(f"💡 Only {len(df)} pairs available - need at least 10 for chart")

🎨 Creating volume chart for Telegram...


💾 Generating PNG chart for Telegram...


I0000 00:00:1757511449.926098 6612158 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


✅ PNG chart saved: /tmp/volume_chart.png
📤 Sending PNG chart via Telegram...
  📊 PNG Chart: ✅ Sent successfully!
🎉 Volume chart delivered to Telegram!


# 📋 Advanced: Detailed CSV Reports
# Generate and send comprehensive data files

In [8]:
# 📊 Generate and Send Comprehensive CSV Report
# Create detailed data export for advanced analysis

if not df.empty:
    print("📋 Generating detailed CSV report...")
    
    # Prepare comprehensive dataset
    detailed_df = df.copy()
    
    # Add timestamp for the report
    detailed_df['report_timestamp'] = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')
    
    # Calculate additional derived metrics
    detailed_df['volume_24h_usd_millions'] = detailed_df['volume_24h_usd'] / 1_000_000
    detailed_df['market_cap_rank'] = detailed_df['volume_rank']  # Use volume rank as proxy
    
    # Add performance categories
    detailed_df['performance_category'] = detailed_df['price_change_24h_pct'].apply(
        lambda x: 'Strong Gain' if x > 5 else 
                  'Moderate Gain' if x > 1 else 
                  'Stable' if x > -1 else 
                  'Moderate Loss' if x > -5 else 'Strong Loss'
    )
    
    # Add volume categories
    volume_quantiles = detailed_df['volume_24h_usd'].quantile([0.25, 0.5, 0.75])
    detailed_df['volume_category'] = detailed_df['volume_24h_usd'].apply(
        lambda x: 'Very High' if x > volume_quantiles[0.75] else
                  'High' if x > volume_quantiles[0.5] else
                  'Medium' if x > volume_quantiles[0.25] else 'Low'
    )
    
    # Reorder columns for better presentation
    column_order = [
        'volume_rank', 'trading_pair', 'connector_name', 'price', 
        'volume_24h_usd', 'volume_24h_usd_millions', 'volume_category',
        'price_change_24h_pct', 'performance_category', 'volatility_pct',
        'high_24h', 'low_24h', 'position_in_range', 'volume_trend_pct',
        'composite_score', 'data_points', 'report_timestamp'
    ]
    
    # Add any remaining columns
    remaining_cols = [col for col in detailed_df.columns if col not in column_order]
    final_columns = column_order + remaining_cols
    detailed_df = detailed_df[final_columns]
    
    # Create timestamped filename
    timestamp_str = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M")
    csv_path = f'/tmp/volume_screener_report_{timestamp_str}.csv'
    
    # Save to CSV with proper formatting
    detailed_df.to_csv(csv_path, index=False, float_format='%.8f')
    
    print(f"✅ CSV report generated: {len(detailed_df)} records")
    print(f"💾 File location: {csv_path}")
    
    # Send via Telegram if available
    telegram_notifier = notification_manager.get_notifier('telegram')
    
    if telegram_notifier:
        print("📤 Sending CSV report via Telegram...")
        
        # Create detailed caption
        csv_caption = f"""📋 <b>Detailed Volume Screener Data</b>
📅 {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}
🔗 Exchange: {CONNECTOR_NAME.replace('_', ' ').title()}

📊 Dataset Summary:
• Total Records: {len(detailed_df)}
• Top Volume: ${detailed_df['volume_24h_usd'].max():,.0f}
• Timeframe: {INTERVAL} candles
• Categories: Performance & Volume rankings

🔍 <i>Complete dataset with metrics for advanced analysis</i>"""
        
        try:
            doc_success = await telegram_notifier.send_document(
                csv_path,
                caption=csv_caption
            )
            
            print(f"📋 CSV Report: {'✅ Sent' if doc_success else '❌ Failed'}")
            
            if doc_success:
                print("🎉 Detailed CSV report successfully delivered!")
            else:
                print("⚠️ CSV delivery failed - check Telegram bot configuration")
        except Exception as e:
            print(f"❌ Error sending CSV: {e}")
            print("💾 CSV file saved locally and ready for manual distribution")
            
    else:
        print("⚠️ Telegram notifier not available for CSV sending")
        print(f"💾 CSV file saved locally and ready for manual distribution")
    
    # Display sample of the data
    print(f"\n📋 CSV Report Preview (Top 5 rows):")
    print("-" * 80)
    preview_cols = ['volume_rank', 'trading_pair', 'volume_24h_usd', 'price_change_24h_pct', 'performance_category']
    available_cols = [col for col in preview_cols if col in detailed_df.columns]
    print(detailed_df[available_cols].head().to_string(index=False))
    print("-" * 80)
    
else:
    print("⚠️ No data available for CSV report generation")
    print("💡 Ensure data collection tasks are running and populating the cache")

📋 Generating detailed CSV report...
✅ CSV report generated: 21 records
💾 File location: /tmp/volume_screener_report_20250910_1337.csv
📤 Sending CSV report via Telegram...
📋 CSV Report: ✅ Sent
🎉 Detailed CSV report successfully delivered!

📋 CSV Report Preview (Top 5 rows):
--------------------------------------------------------------------------------
 volume_rank trading_pair      volume_24h_usd  price_change_24h_pct performance_category
           1     ETH-USDT 4380802255.90790939             0.7973148               Stable
           2     BTC-USDT 3864052879.80800009            0.35790649               Stable
           3     SOL-USDT 1624360790.44979978            1.17154812        Moderate Gain
           4     WLD-USDT      1260812639.648           12.32217115          Strong Gain
           5     XRP-USDT     573312999.77616            1.44076661        Moderate Gain
--------------------------------------------------------------------------------


# 🧹 Cleanup and Finalization
# Clean up resources and temporary files

In [9]:
# 🔒 Cleanup and Summary
# Clean up temporary files and provide execution summary

print("✅ Volume screener analysis completed successfully")

# Optional: Clean up temporary files
import os
temp_files = ['/tmp/volume_chart.html', '/tmp/volume_chart.png']
csv_files = [f for f in os.listdir('/tmp') if f.startswith('volume_screener_report_') and f.endswith('.csv')]

cleanup_count = 0
for file_path in temp_files:
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            cleanup_count += 1
            print(f"🗑️  Cleaned up: {file_path}")
        except Exception as e:
            print(f"⚠️  Could not remove {file_path}: {e}")

# Optionally clean up old CSV files (keep only the most recent)
if len(csv_files) > 1:
    csv_files_full = [f'/tmp/{f}' for f in csv_files]
    csv_files_full.sort(key=lambda x: os.path.getctime(x))
    
    # Remove all but the most recent
    for old_csv in csv_files_full[:-1]:
        try:
            os.remove(old_csv)
            cleanup_count += 1
            print(f"🗑️  Cleaned up old CSV: {os.path.basename(old_csv)}")
        except Exception as e:
            print(f"⚠️  Could not remove {old_csv}: {e}")

if cleanup_count > 0:
    print(f"🧹 Cleaned up {cleanup_count} temporary files")
else:
    print("🧹 No temporary files to clean up")

# Execution Summary
print("\n📊 EXECUTION SUMMARY")
print("=" * 50)
print(f"📈 Data Source: CLOB Cached Data ({CONNECTOR_NAME})")
print(f"⏰ Timeframe: {INTERVAL} candles")
print(f"💹 Pairs Analyzed: {len(df) if not df.empty else 0}")
print(f"📤 Report Sent: {'✅ Yes' if enabled_notifiers else '❌ No notifiers configured'}")
print(f"📊 Charts Generated: {'✅ Yes' if not df.empty and len(df) >= 10 else '❌ Insufficient data'}")
print(f"📋 CSV Export: {'✅ Yes' if not df.empty else '❌ No data available'}")

print("\n🎉 Volume screener report generation complete!")
if 'telegram' in enabled_notifiers:
    print("📱 Check your Telegram for delivered reports, charts, and data files")
else:
    print("💡 Configure Telegram in .env to receive automated reports")

print("\n🔧 Next Steps:")
print("  • Run data collection tasks regularly for fresh data")
print("  • Adjust TOP_PAIRS_COUNT for different report sizes")
print("  • Set FETCH_FRESH_DATA = True for real-time analysis")
print("  • Schedule this notebook as a task for automated reports")

✅ Volume screener analysis completed successfully
🗑️  Cleaned up: /tmp/volume_chart.html
🗑️  Cleaned up: /tmp/volume_chart.png
🧹 Cleaned up 2 temporary files

📊 EXECUTION SUMMARY
📈 Data Source: CLOB Cached Data (binance_perpetual)
⏰ Timeframe: 15m candles
💹 Pairs Analyzed: 21
📤 Report Sent: ✅ Yes
📊 Charts Generated: ✅ Yes
📋 CSV Export: ✅ Yes

🎉 Volume screener report generation complete!
📱 Check your Telegram for delivered reports, charts, and data files

🔧 Next Steps:
  • Run data collection tasks regularly for fresh data
  • Adjust TOP_PAIRS_COUNT for different report sizes
  • Set FETCH_FRESH_DATA = True for real-time analysis
  • Schedule this notebook as a task for automated reports


# 🛠️ Setup and Configuration Guide

## 📱 Telegram Bot Configuration

### 1. Create Telegram Bot
1. **Start BotFather**: Message @BotFather on Telegram
2. **Create Bot**: Send `/newbot` and follow prompts
3. **Get Token**: Save the bot token (format: `123456789:ABCDEFGHIJKLMNOPQRSTUVWXYZ`)
4. **Get Chat ID**: 
   - Start conversation with your bot
   - Message @userinfobot to get your chat ID
   - Or use group chat ID for team notifications

### 2. Environment Variables
Add these to your `.env` file:
```bash
# Telegram Configuration
TELEGRAM_ENABLED=true
TELEGRAM_BOT_TOKEN=your_bot_token_here
TELEGRAM_CHAT_ID=your_chat_id_here
TELEGRAM_PARSE_MODE=HTML
TELEGRAM_DISABLE_NOTIFICATION=false
```

### 3. Data Collection Setup
Ensure you have cached data by running:
```bash
# Run data collection tasks to populate cache
python cli.py run app.tasks.data_collection.candles_downloader_task

# Or fetch data directly in notebook by setting:
FETCH_FRESH_DATA = True
```

## 🚀 Running the Report

### Manual Execution
1. **Activate Environment**: `conda activate quants-lab`
2. **Start Jupyter**: `jupyter lab`
3. **Run Notebook**: Execute all cells in order
4. **Check Telegram**: Reports delivered to configured chat

### Automated Execution
Convert to scheduled task by creating a task configuration:
```yaml
# In config/telegram_report.yml
version: '2.0'

tasks:
  telegram_volume_report:
    enabled: true
    task_class: notebook
    schedule:
      type: frequency
      frequency_hours: 24  # Daily reports
    config:
      notebooks:
        - research_notebooks/market_data_etl/telegram_screener_report.ipynb
      parameters:
        TOP_PAIRS_COUNT: 15
        INTERVAL: "1h"
        FETCH_FRESH_DATA: false
```

## 🔧 Troubleshooting

### Common Issues
- **No Data**: Set `FETCH_FRESH_DATA = True` or run data collection tasks
- **Telegram Fails**: Verify bot token and chat ID in .env
- **Chart Generation**: Install `kaleido` for PNG export: `pip install kaleido`
- **Cache Empty**: Run candles downloader task first

### Verification Steps
1. Check cached data: `len(clob.candles_cache)` should be > 0
2. Test Telegram bot: Send test message manually
3. Verify notification system: Check enabled notifiers
4. Ensure dependencies: `pip install plotly kaleido pandas`

### Configuration Options
- **TOP_PAIRS_COUNT**: Number of pairs in report (default: 15)
- **INTERVAL**: Candle timeframe ("1m", "5m", "1h", "1d")
- **CONNECTOR_NAME**: Exchange to analyze ("binance_perpetual", etc.)
- **QUOTE_ASSET**: Quote currency filter ("USDT", "BTC", etc.)
- **FETCH_FRESH_DATA**: True to fetch from API, False to use cache